In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers pyngrok python-dotenv streamlit

In [2]:
from threading import Thread
from pyngrok import ngrok
import os
ngrok_token = '2otPVOrLKg1IybDuNrBzRMcKLtd_LihT3pBrbkvhZBGCFK1G'
ngrok.set_auth_token(ngrok_token)

In [3]:
def run_streamlit():
    os.system("streamlit run /content/app.py --server.port 8501")

In [13]:
%%writefile app.py
import streamlit as st
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
from openai import OpenAI

st.set_page_config(
    page_title="CodeBase Assistant",
    page_icon="🤖"
)

@st.cache_resource
def initialize_components():
    try:
        pinecone_api_key = "pcsk_vxaeh_Ket2P4K4LjgvR8taWp5LmDXaDJvZjYpKeCAPKo99bdfSrnHc6gcxh9RzbBKGA1r"
        groq_api_key = "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"

        pc = Pinecone(api_key=pinecone_api_key)
        pinecone_index = pc.Index("codebase-rag")

        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=groq_api_key
        )

        model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

        return pinecone_index, client, model
    except Exception as e:
        st.error(f"Error initializing components: {str(e)}")
        return None, None, None

def get_answer(query: str, pinecone_index, client, model):
    try:
        # Get query embeddings
        query_embedding = model.encode(query)

        # Query Pinecone
        results = pinecone_index.query(
            vector=query_embedding.tolist(),
            top_k=5,
            include_metadata=True,
            namespace="https://github.com/CoderAgent/SecureAgent"
        )

        contexts = [item['metadata']['text'] for item in results['matches']]

        prompt = f"""
        Context: {' '.join(contexts)}

        Question: {query}
        """

        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": "You are a helpful code assistant."},
                {"role": "user", "content": prompt}
            ]
        )

        return response.choices[0].message.content
    except Exception as e:
        st.error(f"Error: {str(e)}")
        return None

def main():
    st.title("CodeBase Assistant")

    # Initialize components
    pinecone_index, client, model = initialize_components()

    if not all([pinecone_index, client, model]):
        st.error("Please check your API keys in the Streamlit secrets.")
        return

    # Query input
    query = st.text_area("Ask your question:", height=100)

    if st.button("Get Answer"):
        if query:
            with st.spinner("Getting answer..."):
                response = get_answer(query, pinecone_index, client, model)
                if response:
                    st.markdown("### Answer:")
                    st.write(response)
        else:
            st.warning("Please enter a question.")

main()

Overwriting app.py


In [6]:
thread = Thread(target=run_streamlit)
thread.start()

In [7]:
public_url = ngrok.connect(addr="8501", proto="http", bind_tls=True)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://791d-34-19-103-155.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
    print(f"Closing tunnel: {tunnel.public_url} -> {tunnel.config['addr']}")
    ngrok.disconnect(tunnel.public_url)